In [2]:
using Pkg

  Updating git-repo `https://github.com/diegozea/ROC.jl`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [3]:
using JuMP, Gurobi, CSV, LinearAlgebra, DataFrames, Random, Distributions, Statistics,MLBase, CPUTime,ScikitLearn ,MLDataUtils
using ScikitLearn: fit!, predict
@sk_import metrics: roc_auc_score
@sk_import linear_model: LogisticRegression
gurobi_env = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x00007f8a52d09600)

In [5]:
function one_hot_encode(X, names)
    X2 = deepcopy(X)
    select!(X2, Not(Symbol.(names)))
    for i in names
        vales = unique(X[i])
        for j in 1:length(vales)-1
           X2[Symbol(string(i)*"_"*string(vales[j]))] = (X[i].==vales[j])*1
        end
    end
    return X2
end

function normalize(X, names)
    X2 = deepcopy(X)
    select!(X2, Not(Symbol.(names)))
    for j in names
        X2[j] = (X[:,j] .- mean(X[:,j])) / std(X[:,j])
    end
    return X2
end

function clean(X)
    n,p = size(X)
    i = 0
    while i < n
        i += 1
        if "?" in X[i,:]
            X = X[1:end .!= i, :]
            i -= 1
        end
        n,_ = size(X)
    end
    return X
end     

function toNum(df, names)
    n,p = size(df)
    for name in names
        if !(isa(df[1,name], Int64) || isa(df[1,name], Float64))
            temp = zeros(n)
            for i=1:n
                temp[i] = parse(Float64,df[i,name])
            end
            df[!,name] = temp
        end
    end
    return df
end

function preprocess(df, categorical_vars, numerical_vars)
    df = clean(df)
    df = toNum(df,numerical_vars)
    df = normalize(df,numerical_vars)
    df = one_hot_encode(df[:,1:end], categorical_vars) 
    df[df[:,end].==0,end] .= -1
    return df
end

preprocess (generic function with 1 method)

In [6]:
### Utils Functions ###
# function compute_∇f(w_k, y, X, λ)
#     n, p = size(X)
#     temp = zeros(p)
#     for i in 1:n
#         expo = log(1+exp(-y[i]*dot(X[i,:], w_k)+ λ*dot(w_k,w_k)))
#         if expo == Inf
#             expo = -y[i]*dot(X[i,:], w_k)+ λ*dot(w_k,w_k)
#         end
#          Δ = (1/(1+t))*t*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
# #        Δ = (-1/(1+exp(y[i]*dot(w_k,X[i,:])+λ*transpose(w_k)*w_k)))*(y[i].*X[i,:] .+ 2*λ*w_k)
#         temp = temp + Δ

#     end

#     ∇f_k = temp
#     return ∇f_k
# end
function compute_∇f(w_k, y, X, λ)
    n, p = size(X)
    temp = zeros(p)
    for i in 1:n
        t = exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k)
#       t = exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k)

        if t == Inf
            Δ = -y[i]*Array(X[i,:]) .+ 2*λ*w_k
        else
            Δ = (1/(1+t))*t*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
        end
        temp = temp + Δ
    end
    ∇f_k = temp
    return ∇f_k
end

function compute_f(w_k, y, X, λ)
    n, p = size(X)
    taylor = 0
    f_k = 0
    for i in 1:n
        t = exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)
        if t == Inf 
            f_k += -y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k
        else
            f_k += log(1+t)
        end
    end

    return f_k
end

compute_f (generic function with 1 method)

In [7]:
### Cutting Planes Implementation ###
function LR_cutting_planes(y, X, ε, λ)
    errors = []
    n, p = size(X)
    # Initialization values and step 0
    w_0 = [0 for i in 1:p]
    #w_0 = [rand(Uniform(-0.5, 0.5)) for i in 1:p]
    f_0 = sum(log(1+exp(-y[i]*dot(X[i,:], w_0)+λ*transpose(w_0)*w_0)) for i=1:n)
    ∇f_0 = compute_∇f(w_0, y, X, λ)

    # Outer minimization problem
    outer_min_model = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
    @variable(outer_min_model, t >= 0)
    @variable(outer_min_model, w[1:p])
    #@constraint(outer_min_model, [j=1:p], -1 <= w[j] <= 1)
    @constraint(outer_min_model, t >= f_0 + (dot(∇f_0, w)-dot(∇f_0, w_0)))
    @constraint(outer_min_model, [j=1:p], 10 >= w[j])
    @constraint(outer_min_model, [j=1:p], w[j] >= -10)
    @objective(outer_min_model, Min, t)
    k = 1 # Number of constraints in the final problem
    solve(outer_min_model)

    # New steps k
    t_k = getvalue(t)
    w_k = getvalue(w)

    f_k = compute_f(w_k, y, X, λ)
    #f_k = sum(min(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)),taylor) for i=1:n)
    
    ∇f_k = compute_∇f(w_k, y, X, λ)
    
    while abs(f_k - t_k) >= ε # error

        push!(errors, f_k - t_k)
        @constraint(outer_min_model,t >= f_k +(dot(∇f_k, w)-dot(∇f_k, w_k)))
        k += 1
        solve(outer_min_model)
        # Updating all the values
        t_k = getvalue(t)
        w_k = getvalue(w)
        
        f_k = compute_f(w_k, y, X, λ)
        #f_k = sum(min(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)),100) for i=1:n)

        ∇f_k = compute_∇f(w_k, y, X, λ)
         if k%500 == 0
             println("Number of constraints: ", k, "\t Error = ", abs(t_k - f_k))
         end
        if k > 20000
            break
        end
    end
    push!(errors, f_k - t_k)
    return t_k, f_k, w_k, errors
end


LR_cutting_planes (generic function with 1 method)

In [8]:
function robust_LG_cv(X, y, ε, lambda_vals; method=LR_cutting_planes, nfold = 5)
    n,p = size(X)
    permuted_indices = randperm(n)
    assignments = repeat(1:nfold, convert(Int,ceil(n/nfold)))[1:n]
    AUCs = zeros(length(lambda_vals))
    for fold in 1:nfold
        println("nfold=",fold)
        train_indices, valid_indices = permuted_indices[assignments.!=fold], permuted_indices[assignments.==fold]

        X_train, y_train = X[train_indices,:], y[train_indices]
        X_valid, y_valid = X[valid_indices,:], y[valid_indices] 
         
        for (i,λ) in enumerate(lambda_vals)
            println(i)
            t, f, w, e = method(y_train, X_train, ε, λ)
    #         pred = 1 ./ (1 .+ exp.(-(Matrix(X_valid)*w).+λ*transpose(w)*w)) .> 0.5
    #         accuracies[i] = 1-sum(pred .!= (y_valid .== 1))/length(y_valid)
            pred_prob = 1 ./ (1 .+ exp.(-(Matrix(X_valid)*w).+λ*transpose(w)*w))
            AUCs[i] += roc_auc_score(y_valid ,pred_prob)
        end
    end
    IJulia.clear_output()

    i_best = argmax(AUCs)
    t, f, w_best, e = method(y, X, ε, lambda_vals[i_best])
    return w_best, lambda_vals[i_best]
end

robust_LG_cv (generic function with 1 method)

In [25]:
function robust_LG_valid(X, y, ε, lambda_vals; method=LR_cutting_planes, split_at=0.8)
    n,p = size(X)
    split = convert(Int,floor(split_at*n))
    
    permuted_indices = randperm(n)
    train_indices, valid_indices = permuted_indices[1:split], permuted_indices[split+1:end]
    X_train, y_train = X[train_indices,:], y[train_indices]
    X_valid, y_valid = X[valid_indices,:], y[valid_indices]
    
    AUCs = zeros(length(lambda_vals))
    for (i,λ) in enumerate(lambda_vals)
        println(i)
        t, f, w, e = method(y_train, X_train, ε, λ)
#         pred = 1 ./ (1 .+ exp.(-(Matrix(X_valid)*w).+λ*transpose(w)*w)) .> 0.5
#         accuracies[i] = 1-sum(pred .!= (y_valid .== 1))/length(y_valid)
        pred_prob = 1 ./ (1 .+ exp.(-(Matrix(X_valid)*w).+λ*transpose(w)*w))
        AUCs[i] = roc_auc_score(y_valid ,pred_prob)
    end
    IJulia.clear_output()

    i_best = argmax(AUCs)
    t, f, w_best, e = method(y, X, ε, lambda_vals[i_best])
    return w_best, lambda_vals[i_best]
end

robust_LG_valid (generic function with 1 method)

In [79]:
function train_test_split(df, test_split, seed)
    seed = 1
    split = 0.8
    Random.seed!(seed)
    n,_ = size(df)
    df1 = df[df[:,end].==1,:]
    df2 = df[df[:,end].==-1,:]
    n,_ = size(df1)
    permuted_indices = randperm(n)
    test_split = convert(Int,floor(split*n))
    train_indices, test_indices = permuted_indices[1:test_split], permuted_indices[test_split+1:end]
    train1 = df1[train_indices,:]
    test1 = df1[test_indices,:]
    n,_ = size(df2)
    permuted_indices = randperm(n)
    test_split = convert(Int,floor(split*n))
    train_indices, test_indices = permuted_indices[1:test_split], permuted_indices[test_split+1:end]
    train2 = df2[train_indices,:]
    test2 = df2[test_indices,:]

    train_df = vcat(train1, train2)
    test_df = vcat(test1, test2)
#     train_X = train[:,1:end-1]
#     train_y = train[:,end]
#     test_X = test[:,1:end-1]
#     test_y = test[:,end]
    IJulia.clear_output()
    return train_df, test_df
end

train_test_split (generic function with 1 method)

In [11]:
function test_cv(train, test, ε, lambda_vals, cv, seed)
    
    Random.seed!(seed)
    
    train_y = train[:,end] 
    train_X = train[:,1:end-1] 
    test_y = test[:,end] 
    test_X = test[:,1:end-1] 
    
    start = time()
    w, λ = robust_LG_cv(train_X, train_y, ε, lambda_vals; method=LR_cutting_planes, nfold=cv)
    elapsed = time() - start
    
    pred_prob = 1 ./ (1 .+ exp.(-(Matrix(test_X)*w) .+ λ*transpose(w)*w))
    pred = pred_prob.> 0.5
    accuracy = 1-sum(pred .!= (test_y .== 1))/length(test_y)
    auc = roc_auc_score(test_y ,pred_prob)
    IJulia.clear_output()

    return auc, accuracy, λ, elapsed, w
end

test_cv (generic function with 1 method)

In [12]:
function test(train, test, validation_split, ε, lambda_vals, seed)
    Random.seed!(seed)

    train_y = train[:,end] 
    train_X = train[:,1:end-1] 
    test_y = test[:,end] 
    test_X = test[:,1:end-1] 
    
    start = time()
    w, λ, errors  = robust_LG_valid(train_X, train_y, ε, lambda_vals; method=LR_cutting_planes, split_at=validation_split)
    elapsed = time() - start
    pred_prob = 1 ./ (1 .+ exp.(-(Matrix(test_X)*w).+λ*transpose(w)*w))
    pred = pred_prob.> 0.5
    accuracy = 1-sum(pred .!= (test_y .== 1))/length(test_y)
    auc = roc_auc_score(test_y ,pred_prob)
    IJulia.clear_output()

    return auc, accuracy, λ, elapsed, w
end

test (generic function with 1 method)

# 1. Caesarian (n=80, p=6)

In [80]:
df = CSV.read("Data/caesarian.csv";header=true)
size(df)

(80, 6)

In [81]:
lambda_vals = [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.15]
categorical_vars = Symbol.(["Delivery number" ;"Delivery time";"Blood of Pressure";"Heart Problem"])
numerical_vars = Symbol.(["Age"])
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(80, 10)

In [82]:
seed = 1
test_split = 0.8
train_df, test_df = train_test_split(df, test_split, seed)
;

In [83]:
sum(train_df[:,end].==1)

40

In [84]:
sum(test_df[:,end].==1)

10

In [85]:
CSV.write("Data/caesarian_train.csv", train_df)
CSV.write("Data/caesarian_test.csv", test_df)
;

In [1280]:
ε = 0.0001
validation_split = 0.75
auc, acc, lambda, elapsed, w = test(train_df, test_df, validation_split,ε, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.7676767676767677 Accuracy: 0.6 λ: 0.1 Time: 1.5931670665740967

In [1]:
cv = 5
auc, acc, lambda, elapsed, w  = test_cv(train_df, test_df, ε, lambda_vals, cv, seed)
print("5-fold: ", "AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

UndefVarError: UndefVarError: test_cv not defined

# 2. Monks (n=432, p=7)

In [95]:
df = CSV.read("Data/monks-1.test";header=false)[:,2:end]
select!(df, Not(Symbol.("Column9")))
df[!, 1], df[!, end] = df[!, end], df[!, 1]
size(df)

(432, 7)

In [96]:
lambda_vals = [0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.15]
categorical_vars = propertynames(df)
numerical_vars = []
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(432, 12)

In [97]:
seed = 1
test_split = 0.8
train_df, test_df = train_test_split(df, test_split, seed)
CSV.write("Data/monks_train.csv", train_df)
CSV.write("Data/monks_test.csv", test_df)
;

In [1225]:
ε = 0.0001
validation_split = 0.75
auc, acc, lambda, elapsed, w = test(df, train_indices, test_indices,validation_split,ε, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.7127586206896552 Accuracy: 0.8055555555555556 λ: 0.0001 Time: 9.801400184631348

In [1226]:
cv = 5
auc, acc, lambda, elapsed, w  = test_cv(df, train_indices, test_indices, test_split, ε, lambda_vals, cv, seed)
print("5-fold: ", "AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

5-fold: AUC: 0.7341379310344828 Accuracy: 0.8055555555555556 λ: 0.05 Time: 47.951651096343994

In [104]:
df = CSV.read("Data/credit-screening/crx.data";header=false)
n,p=size(df)

(690, 16)

In [105]:
categorical_vars = propertynames(df[1,vcat(1,4:7,9:10,12:13,16)])
numerical_vars = propertynames(df[1,vcat(2:3,8,11,14:15)])
lambda_vals = [0.0001,0.0005,0.001,0.005,0.01]
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(653, 38)

In [106]:
seed = 1
test_split = 0.8
train_df, test_df = train_test_split(df, test_split, seed)
CSV.write("Data/credit_screening_train.csv", train_df)
CSV.write("Data/credit_screening_test.csv", test_df)
;

In [26]:
ε = 0.0001
validation_split = 0.75
auc, acc, lambda, elapsed, w = test(train_df, test_df, validation_split,ε, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

Number of constraints: 500	 Error = 1439.6292715421812
Number of constraints: 1000	 Error = 654.4704112321704
Number of constraints: 1500	 Error = 420.46799008266663
Number of constraints: 2000	 Error = 36.14944384396924
Number of constraints: 2500	 Error = 1.3031137488476645
Number of constraints: 3000	 Error = 0.0865058902080591
Number of constraints: 3500	 Error = 0.0028210480206496413
Number of constraints: 4000	 Error = 0.0001886960461945364


BoundsError: BoundsError: attempt to access ([0.0578455, 0.0524465, 0.432811, 0.565377, -0.320367, 1.29095, -0.214306, -0.508425, -0.826341, -0.507486, -0.826279, 0.224579, -0.0202637, -0.241719, 0.00558883, 0.665868, -0.418377, -0.311449, 0.0787315, 0.871551, -0.455002, 0.0904935, -0.438984, -0.77533, -0.118963, 0.159474, -0.222288, -0.520447, 0.0688339, -0.424366, -0.0399793, 0.0733568, 2.97943, 0.672507, 0.0950756, -0.74015, -0.281947], 0.01)
  at index [3]

In [1187]:
#5-fold cv
seed = 1
auc, acc, lambda, elapsed, w = test_cv(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, 5, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.9189349992562845 Accuracy: 0.8719512195121951 λ: 0.01 Time: 1375.2192559242249

In [1134]:
seed = 2
auc, acc, lambda, elapsed,w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.9585037104346509 Accuracy: 0.8841463414634146 λ: 0.005 Time: 302.3048839569092

In [1135]:
seed = 3
auc, acc, lambda, elapsed,w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.9219504830917874 Accuracy: 0.8658536585365854 λ: 0.01 Time: 361.72257113456726

In [1136]:
maximum(w)

3.0625289434121163

In [1266]:
df = CSV.read("Data/framingham.csv";header=true)
size(df)

(3658, 16)

In [1267]:
categorical_vars = Symbol.(["education"])
numerical_vars = Symbol.(["age" ;"cigsPerDay";"totChol";"sysBP";"diaBP";"BMI";"heartRate";"glucose"])
lambda_vals = [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1]
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(3658, 18)

In [1268]:
seed = 1
test_split = 0.8
train_df, test_df = train_test_split(df, test_split, seed)
CSV.write("Data/framingham_train.csv", train_df)
CSV.write("Data/framingham_test.csv", test_df)
;

In [1139]:
seed = 1
auc, acc, lambda,elapsed,w = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.0001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.8933895907323683 Accuracy: 0.8273224043715847 λ: 0.001 Time: 227.89117217063904

In [1188]:
#5-fold cv
seed = 1
auc, acc, lambda, elapsed, w = test_cv(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, 5, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.9189349992562845 Accuracy: 0.8719512195121951 λ: 0.01 Time: 1408.6571869850159

In [1140]:
seed = 2
auc, acc, lambda,elapsed = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.0001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.8835407884949168 Accuracy: 0.8185792349726776 λ: 1.0e-5 Time: 229.15803718566895

In [1141]:
seed = 3
auc, acc, lambda,elapsed = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.0001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.9007934540044631 Accuracy: 0.8360655737704918 λ: 1.0e-5 Time: 228.73048520088196

In [1269]:
df = CSV.read("Data/default of credit card clients.csv";header=true)
n,p = size(df)

(30000, 25)

In [1270]:
propertynames(df[1,vcat(4:5)])

2-element Array{Symbol,1}:
 :EDUCATION
 :MARRIAGE 

In [1271]:
categorical_vars = propertynames(df[1,vcat(4:5)])
numerical_vars = propertynames(df[1,vcat([2,6],7:24)])
lambda_vals = [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05]
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(30000, 32)

In [1272]:
seed = 1
test_split = 0.8
train_df, test_df = train_test_split(df, test_split, seed)
CSV.write("Data/default_creditcard_train.csv", train_df)
CSV.write("Data/default_creditcard_test.csv", test_df)
;

In [1145]:
seed = 1
auc, acc, lambda,elapsed,w = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.0001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

Enter cross-validation
1
Number of constraints: 500	 Error = 3034.147460961999
Number of constraints: 1000	 Error = 2038.3273531629804
Number of constraints: 1500	 Error = 757.9373835279533
Number of constraints: 2000	 Error = 1099.0112429838846
Number of constraints: 2500	 Error = 1068.8599842678923
Number of constraints: 3000	 Error = 1037.9689351804584
Number of constraints: 3500	 Error = 1446.651848430472
Number of constraints: 4000	 Error = 1085.485219207882
Number of constraints: 4500	 Error = 656.6829614483964
Number of constraints: 5000	 Error = 727.3618853526762
Number of constraints: 5500	 Error = 1871.9293495737795
Number of constraints: 6000	 Error = 538.0591961111122
Number of constraints: 6500	 Error = 576.5444218938436
Number of constraints: 7000	 Error = 491.20520704892573
Number of constraints: 7500	 Error = 483.3426844158958
Number of constraints: 8000	 Error = 572.726774638979
Number of constraints: 8500	 Error = 475.50508825754514
Number of constraints: 9000	 Error 

InterruptException: InterruptException:

In [1146]:
w

17-element Array{Float64,1}:
  0.7564803178300495 
  0.6510781125823433 
  0.05577898804498388
  0.25542433043219676
  0.25844451713654576
  0.886531016636607  
  0.328422611020675  
  0.30110156571079594
 -0.3114629158611582 
 -0.10614015823341487
  0.16820402648139052
 -0.27178104345394566
  0.366384084231571  
  0.11367239852267316
 -0.17436851270711196
 -4.9729848625886035 
 -5.615896549915456  